In [1]:
import os
import numpy as np
import tensorflow as tf

tf.set_random_seed(1)
np.random.seed(1)

/Users/Sylvain/.pyenv/versions/3.6.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Input data

In [2]:
from data_utils import train_cnn_input_fn, test_cnn_input_fn

In [3]:
class_nbr = 6
batch_size = 2

In [4]:
train_input_fn = lambda: train_cnn_input_fn(class_nbr, "../tmp/cnn_output/VGG16", batch_size, keep_fn=False)
test_input_fn = lambda: test_cnn_input_fn(class_nbr, "../tmp/cnn_output/VGG16", batch_size, keep_fn=False)

In [5]:
# Make sure it works
with tf.Session() as sess: 
    print(sess.run(train_input_fn()))
    print(sess.run(test_input_fn()))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Estimator

In [6]:
from estimator_svdd_naive import naive_svdd_model_fn

In [7]:
n_inputs = 10 # à déterminer
train_steps = 10
model_dir = None

#tf.logging.set_verbosity(tf.logging.DEBUG)

In [8]:
OCClassifier = tf.estimator.Estimator(
    model_fn=naive_svdd_model_fn,
    params={
        "frac_err": 50,
        "n_inputs": n_inputs,
        "kernel": "linear",
        "rffm_dims": 200,
        "rffm_stddev": 25,
        "learning_rate": 0.1,
        "input_size": 25088
    },
    model_dir=model_dir
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/_f/7t_whmpx0zn2ln3npht19g8h0000gn/T/tmp3e6_hhhn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x118f8a710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train

In [9]:
OCClassifier.train(
    input_fn=train_input_fn,
    steps=train_steps
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/_f/7t_whmpx0zn2ln3npht19g8h0000gn/T/tmp3e6_hhhn/model.ckpt.
INFO:tensorflow:loss = 3542.1301, step = 1
INFO:tensorflow:Saving checkpoints for 10 into /var/folders/_f/7t_whmpx0zn2ln3npht19g8h0000gn/T/tmp3e6_hhhn/model.ckpt.
INFO:tensorflow:Loss for final step: 2695.3157.


## Test

In [10]:
predictions = OCClassifier.predict(
    input_fn=test_input_fn
)
predictions_list = list(predictions)
predictions_distance = np.asarray(list(map(lambda p: p["predicted_distance"], predictions_list)))
predictions_classes = np.asarray(list(map(lambda p: p["predicted_classes"], predictions_list)))
mapped_inputs = np.asarray(list(map(lambda p: p["mapped_inputs"], predictions_list)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/_f/7t_whmpx0zn2ln3npht19g8h0000gn/T/tmp3e6_hhhn/model.ckpt-10
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [15]:
predictions_classes.shape

(300,)

In [11]:
x = tf.placeholder(tf.float32, shape=None)

In [12]:
x.get_shape().dims is None#tf.TensorShape((None))#.as_list()

True